In [2]:
#https://console.bluemix.net/docs/services/cloud-object-storage/libraries/python.html#python
!pip install ibm-cos-sdk

    100% |████████████████████████████████| 51kB 7.1MB/s eta 0:00:01
    100% |████████████████████████████████| 972kB 860kB/s eta 0:00:01
    100% |████████████████████████████████| 61kB 8.1MB/s eta 0:00:01
  Running setup.py bdist_wheel for ibm-cos-sdk ... done
  Stored in directory: /gpfs/fs01/user/sca9-7277eb31bca08b-bc196c953de3/.cache/pip/wheels/5a/66/38/325cdc3e6a46a409f3a95f89855d1c56e944b7cfa82836f2f2
  Running setup.py bdist_wheel for ibm-cos-sdk-core ... done
  Stored in directory: /gpfs/fs01/user/sca9-7277eb31bca08b-bc196c953de3/.cache/pip/wheels/21/a8/30/72f8035afef96e79bd36919ce8af00045b8bea861d3692c858
Successfully built ibm-cos-sdk ibm-cos-sdk-core


In [3]:
import boto3
import json
import requests
import random
from botocore.client import Config
from pprint import pprint

In [5]:
credentials = json.loads('{"apikey": "2SozF9MkHGQULJZHZTiZOnidaLSc3zIqr3SkDUC0YD0t", "endpoints": "https://cos-service.bluemix.net/endpoints", "resource_instance_id": "crn:v1:bluemix:public:cloud-object-storage:global:a/db0d062d2b4c0836e18618a5222d8068:22e3b946-6154-4032-8e8f-7cfb0b429602::"}')

In [8]:
# Rquest detailed enpoint list
endpoints = requests.get(credentials.get('endpoints')).json()
print(json.dumps(endpoints, indent=2))
#import pdb; pdb.set_trace()

# Obtain iam and cos host from the the detailed endpoints
iam_host = (endpoints['identity-endpoints']['iam-token'])
cos_host = (endpoints['service-endpoints']['cross-region']['us']['public']['us-geo'])

api_key = credentials.get('apikey')
service_instance_id = credentials.get('resource_instance_id')

# Constrict auth and cos endpoint
auth_endpoint = "https://" + iam_host + "/oidc/token"
service_endpoint = "https://" + cos_host

print("Creating client...")
# Get bucket list
cos = boto3.client('s3',
                    ibm_api_key_id=api_key,
                    ibm_service_instance_id=service_instance_id,
                    ibm_auth_endpoint=auth_endpoint,
                    config=Config(signature_version='oauth'),
                    endpoint_url=service_endpoint)


{
  "identity-endpoints": {
    "iam-token": "iam.bluemix.net",
    "iam-policy": "iampap.bluemix.net"
  },
  "service-endpoints": {
    "regional": {
      "us-south": {
        "public": {
          "us-south": "s3.us-south.objectstorage.softlayer.net"
        },
        "private": {
          "us-south": "s3.us-south.objectstorage.service.networklayer.com"
        }
      },
      "us-east": {
        "public": {
          "us-east": "s3.us-east.objectstorage.softlayer.net"
        },
        "private": {
          "us-east": "s3.us-east.objectstorage.service.networklayer.com"
        }
      }
    },
    "cross-region": {
      "eu": {
        "public": {
          "eu-geo": "s3.eu-geo.objectstorage.softlayer.net",
          "Milan": "s3.mil-eu-geo.objectstorage.softlayer.net",
          "Amsterdam": "s3.ams-eu-geo.objectstorage.softlayer.net",
          "Frankfurt": "s3.fra-eu-geo.objectstorage.softlayer.net"
        },
        "private": {
          "eu-geo": " s3.eu-geo.objectst

In [9]:
# Call S3 to list current buckets
response = cos.list_buckets()

# Get a list of all bucket names from the response
buckets = [bucket['Name'] for bucket in response['Buckets']]

# Print out the bucket list
print("Current Bucket List:")
print(json.dumps(buckets, indent=2))
print("---")

Current Bucket List:
[
  "data-prod",
  "output-prod",
  "streams71aa690a314849ce9b242fd1022a028e",
  "streamsqaf9d8b40caeeb448fae73b536e83caac5",
  "test04"
]
---


In [10]:
bucket_name="streams71aa690a314849ce9b242fd1022a028e"
# Call S3 to list current objects
response = cos.list_objects(Bucket=bucket_name)

# Get a list of all object names from the response
objects = [object['Key'] for object in response['Contents']]

# Print out the object list
print("Objects in %s:" % bucket_name)
print(json.dumps(objects, indent=2))

Objects in streams71aa690a314849ce9b242fd1022a028e:
[
  "notebook/Cos-play_4JEENiMJE.ipynb",
  "notebook/Push-to-MH_VJis2BbJV.ipynb"
]
